### Execution on CIFAR dataset

This notebook aims to reproduce results of the paper: 

*Detection of Adversarial Attacks by Analyzing Deep Features with Multivariate Data Algorithms*


It allows to execute XGBoost on the hidden layers of CIFAR model, and reproduce our results.


For each individual layer and each individual attack:

- it trains XGBoost
- it attempts detection also on the other attacks

Different approaches/optimizations are tested, as described also in the paper. 


It requires the layers available at https://drive.google.com/drive/folders/1JsV45ooRlk5CpqFCPy-uR4iBB3Nbqx08?usp=sharing ,
folder CIFAR


We assume the CIFAR folder is located at path BASE (e.g., BASE=/home/whatever/CIFAR )


We recommend to use the conda environment available on the github.
<br><br>
<br><br>
**For brevity, we consider only the 4 best layers, 156, 105, 20, 88, and we train only on the attacks 'fgsm_0156', 'deep_10'**

This can be changed easily in the second box below.

#### Configure the following items attentively:

In [ ]:
DATASET="cifar"
BATCH_SIZE=128 
EPOCH=30 #useful only for fastai; the model include a stopping criteria so no problem with high Epochs
LOGFILE="cifar_log_0_8.csv" #just a name, choose one
CPU_THREADS=32 # just to limit cpu usage, can be set to any value
RESULTS='/home/notebook/neuron/results/' #the path to the log file
BASE='/home/notebook/neuron/layers/tmp/'  #put your path to CIFAR folder
index_path='/home/notebook/neuron/indexes/' #path to the indexes, as available on google drive. 
                                            #These are required to properly create train and test
    
MODEL_SAVE='/home/notebook/neuron/models_08/' #it will save here the models it creates

From now on, everything should go smoothly (but training can be slow at some times)

The notebook will train the model, execute prediction and log results.

In [ ]:
ATTACK=['bim', 'carlini_l2', 'deep_10', 'fgsm_02', 'fgsm_0156']
SUFFIX='.npy'
MAIN_ATTACK_LIST=['deep_10', 'fgsm_0156']
TOP_LAYERS=[156, 105, 20, 88]

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import sys
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
import csv
from matplotlib import pyplot as plt
import math
import fastai
from fastai.tabular.all import *
from pathlib import Path
from ipywidgets import IntProgress
from IPython.display import display
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import pandas as pd
import time
import torch
from torch import nn

%env OMP_NUM_THREADS=CPU_THREADS
%env MKL_NUM_THREADS=CPU_THREADS

torch.set_num_threads(CPU_THREADS)

Functions to load data

In [ ]:
#load train data for a specific layer
def load_train(L):
    p = Path(BASE+"/train/layer_"+str(L)+".npy")
    with p.open('rb') as f:
        nptmp=np.load(f)
    return nptmp

def load_test(L):
    p = Path(BASE+"/test/layer_"+str(L)+".npy")
    with p.open('rb') as f:
        nptmp=np.load(f)
    return nptmp

#prefix, layer
def load_attack(attack, L):
    p = Path(BASE+"/"+str(attack)+"/layer_"+str(L)+".npy")
    print(p)
    with p.open('rb') as f:
        nptmp=np.load(f)
    return nptmp

def load_4_layers_train(a, b=None, c=None, d=None):
    a=load_train(a)
    if(b!=None):
        b=load_train(b)
    if(c!=None):
        c=load_train(c)
    if(d!= None):
        d=load_train(d)
    return a, b, c, d

def load_4_layers_test(a, b=None, c=None, d=None):
    a=load_test(a)
    if(b!=None):
        b=load_test(b)
    if(c!=None):
        c=load_test(c)
    if(d!= None):
        d=load_test(d)
    return a, b, c, d

def load_4_layers_attack(attack, a=None, b=None, c=None, d=None):
    a=load_attack(attack, a)
    if(b!=None):
        b=load_attack(attack, b)
    if(c!=None):
        c=load_attack(attack, c)
    if(d!= None):
        d=load_attack(attack, d)
    return a, b, c, d

Function to linearize the hidden features for a target layer and image. This allows to apply tabular data algorithms on the hidden features

In [ ]:
def prod(val): 
    res = 1 
    for ele in val: 
        res *= ele 
    return res  

def linearize(numpy_linearized):
    shape_tuple=numpy_linearized.shape[1:]
    row_length=prod(list(shape_tuple))
    numpy_linearized=numpy_linearized.reshape(
        numpy_linearized.shape[0], row_length)
    return numpy_linearized

Prepare to log data

We log:

- METHOD
- how we split  (train/test)
- layer number(s)
- attack for test
- attack used for training
- DATASET (CIFAR)
- metrics: accuracy, tn, fp, fn, tp, 
- tpr when fpr is < 0.05, to compare with evadeML
- tpr when fpr is < 0.015, to compare with MagNet

In [ ]:
def write_log_stacking_split(logfile, METHOD, split,approach, layer, attack, attack1, DATASET, accuracy, tn, fp, fn, tp, tpr1, tpr2):
    logfile.write( "{}, {}, {}, {}, {}, {}, {}, {:.3f}, {}, {}, {}, {}, {:.3f}, {:.3f}\n".format(METHOD,
                                    approach,
                                    split,
                                    layer,
                                    attack,
                                    attack1,
                                    DATASET,
                                    accuracy,
                                    tn, fp, fn, tp,
                                    tpr1, tpr2))
    logfile.flush()


In [ ]:
f = open(RESULTS+LOGFILE, "a")

In [ ]:
f.write("ALG, APPROACH, SPLIT 0.8 or 0.9, LAYER, ATTACK, TRAINED ON, DATASET, ACCURACY, TN, FP, FN, TP, TPR with FPR < 0.05, TPR with FPR < 0.015 \n")
f.flush()

The following creates the train and test datasets.

For example, they match each normal data point to label 0, and each attack data point to label 1.

In [ ]:
def create_normal(normal_x):
    normal_y=np.empty([normal_x.shape[0], 1])
    normal_y.fill(0)
    return normal_x, normal_y

def create_attack(attack_x):
    attack_y=np.empty([attack_x.shape[0], 1])
    attack_y.fill(1)
    return attack_x, attack_y

def create_dataframe(x, y):
    LABEL_NUMBER=x.shape[1]
    df=pd.DataFrame(np.concatenate((x, y), axis=1))
    df[LABEL_NUMBER]=df[LABEL_NUMBER].astype(str)
    cont_names, cat_names = cont_cat_split(pd.DataFrame(x))

    return df, cont_names, cat_names, LABEL_NUMBER

def create_test_set(numpy_test_x,attack_test_x, attack_test_y):
    numpy_test_x=linearize(numpy_test_x)
    numpy_test_y=np.empty([numpy_test_x.shape[0], 1])
    numpy_test_y.fill(0)
    merged_x=np.concatenate((numpy_test_x, attack_test_x), axis=0)
    merged_y=np.concatenate((numpy_test_y, attack_test_y), axis=0)
    df=pd.DataFrame(merged_x)
    return df , merged_y

Now the training and evaluation can start:

- take a layer
- take an attack
- organize a train/test set
- do supervised train using XGBOOST
- do predict on the test set (legitimate images + attack images)
- do predict on all the other attacks


To be fair, we create a test set which includes "normal image + adversarial image", balanced, and both of them have never been seen at training time. This is hard coded here, and cannot be parametrized. 

We provide this approach for a train-test split of 0.8 (default). The user can configure the test split, just modifying obvious parameters.

(Note: the test set may contain additional images, which does not have a corresponding "adversarial part").



In [ ]:
#the index of the "indexes arrays" (see the folder indexes) at which we have the 0.8 train/test split for the under consideration.
#we need it for deep_10 and fgsm_0156
TEST_SPLIT_08=[#('bim', 7873, 'bim_0.008_cifar_index_final.npy'),
               #('carlini_l2', 7059, 'carlini_l2_cifar_index_final.npy'),
               ('deep_10',7824, 'deep_10_cifar_index_final.npy'),
               #('fgsm_02', 0, ''),
               ('fgsm_0156',7059, 'fgsm_0.0156_cifar_index_final.npy')]

TEST_SPLIT=[0.8]


list_combo= [(L, y,d)
             for L in TOP_LAYERS 
             for y in TEST_SPLIT 
             for d in MAIN_ATTACK_LIST
            ]

In [ ]:
def define_attack_set(test_split, attack1, split_value, split_index,
                      x_attack_156,
                      x_attack_105=None,
                      x_attack_20=None,
                      x_attack_88=None):
    if(attack1=="deep_10"):
        SPLIT_PLACE=test_split[2] #deep
    elif(attack1=="fgsm_0156"):
        SPLIT_PLACE=test_split[4] #fgsm_0156

    indexes=np.load(index_path+SPLIT_PLACE[2])
    index_current_attack=np.min(np.where(indexes >= split_index))

    x_attack_156=x_attack_156[index_current_attack:]
    if(x_attack_105 is not None):
        x_attack_105=x_attack_105[index_current_attack:]
    if(x_attack_20 is not None):
        x_attack_20=x_attack_20[index_current_attack:]
    if(x_attack_88 is not None):
        x_attack_88=x_attack_88[index_current_attack:]
    return x_attack_156, x_attack_105, x_attack_20, x_attack_88


def define_attack_set_1(attack1, target_size, x_attack_156,
                      x_attack_105=None,
                      x_attack_20=None,
                      x_attack_88=None):
    if(x_attack_156.shape[0]> target_size):
        index_current_attack=x_attack_156.shape[0]-target_size-1
    else:
        index_current_attack=0

    x_attack_156=x_attack_156[index_current_attack:]
    if(x_attack_105 is not None):
        x_attack_105=x_attack_105[index_current_attack:]
    if(x_attack_20 is not None):
        x_attack_20=x_attack_20[index_current_attack:]
    if(x_attack_88 is not None):
        x_attack_88=x_attack_88[index_current_attack:]

    return x_attack_156, x_attack_105, x_attack_20, x_attack_88


def analyze_split(test_split, attack):
    split="split:"+str(TEST_SPLIT[0])
    split_value=TEST_SPLIT[0]
    #create attack train and attack test
    if(attack=="fgsm_0156"):
        SPLIT_PLACE=TEST_SPLIT_08[1] #fgsm
    elif(attack=="deep_10"): #deepfool
        SPLIT_PLACE=TEST_SPLIT_08[0] #deepfool
#    elif(attack=="bim"): 
#        SPLIT_PLACE=test_split[0]
#    elif(attack=="carlini_l2"): 
#        SPLIT_PLACE=test_split[1]
#    elif(attack=="fgsm_02"): 
#        SPLIT_PLACE=test_split[3]

    split_index_test=SPLIT_PLACE[1]
    return split, split_value, split_index_test

def split_test_normal(x_test_156, test_elements, x_test_105=None, x_test_20=None, x_test_88=None ):
    size=x_test_156.shape[0]
    x_test_156=x_test_156[size-(test_elements+1):]
    if(x_test_105 is not None):
        x_test_105=x_test_105[size-(test_elements+1):]
    if(x_test_20 is not None):
        x_test_20=x_test_20[size-(test_elements+1):]
    if(x_test_88 is not None):
        x_test_88=x_test_88[size-(test_elements+1):]
    return x_test_156,x_test_105, x_test_20, x_test_88


def attack_split(attack_array, split_value):
    x_attack_train=attack_array[0:round(split_value*attack_array.shape[0])]
    x_attack_test=attack_array[round(split_value*attack_array.shape[0]):]
    return x_attack_train, x_attack_test

In [ ]:
#these just computes metrics
def compute_metrics_and_log(y_test, y_attack_test, final_preds, final_preds_proba, approach, METHOD, split, layer):
    accuracy=accuracy_score(np.vstack((y_test, y_attack_test)), final_preds)
    tn, fp, fn, tp = confusion_matrix(np.vstack((y_test, y_attack_test)), final_preds, labels=[0,1]).ravel()
    fpr, tpr, thresholds = metrics.roc_curve(np.vstack((y_test, y_attack_test)), final_preds_proba[:, 1].ravel())
    tpr_05=tpr[np.argmax(fpr[fpr<=0.05])]
    tpr_016=tpr[np.argmax(fpr[fpr<=0.016])]
    write_log_stacking_split(f, METHOD, split, approach, layer, attack, "trained on normal + {}".format(attack), 
                       DATASET, accuracy, tn, fp, fn, tp, tpr_05, tpr_016)

def compute_metrics_and_log_attacks(y_test, y_attack, final_preds, final_preds_proba, approach, METHOD, split, layer):
    accuracy=accuracy_score(y_attack, final_preds)
    tn, fp, fn, tp = confusion_matrix(y_attack, final_preds, labels=[0,1]).ravel()
    fpr, tpr, thresholds = metrics.roc_curve(np.vstack((y_test, y_attack)),
                                                 final_preds_proba[:, 1].ravel())
    #tpr_05=tpr[np.argmax(fpr[fpr<=0.05])] not good when multiple values with the same number
    #tpr_016=tpr[np.argmax(fpr[fpr<=0.016])] not good when multiple values with the same number
    #replaced with:
    
    tpr_05=tpr[np.argwhere(fpr == np.amax(fpr[fpr<=0.05])).flatten()][-1]
    tpr_016=tpr[np.argwhere(fpr == np.amax(fpr[fpr<=0.016])).flatten()][-1]
    write_log_stacking_split(f, METHOD,split, approach, layer, attack1, attack, 
                       DATASET, accuracy, tn, fp, fn, tp, tpr_05, tpr_016)
    
    
    
    roc_auc = metrics.auc(fpr, tpr)

    # method I: plt
    plt.title("layer: {}, trained on {}, tested on {}".format(layer, attack, attack1))
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

Just a plain approach: on each layer, it applies XGBoost and compute metrics.

It saves the models that are later used.


In [ ]:
for layer, test_split, attack in list_combo:
    split, split_value, split_index=analyze_split(test_split, attack)
    #load train 156, 105, 20, 88
    x_train= load_train(layer)
    y_train = np.zeros((x_train.shape[0],1))

    #load attacks of the three layers
    x_attack=load_attack(attack, layer)
    print("{} shape of attacks in train + test  {}".format(attack, x_attack.shape))
    
    #split also attacks, to maintain the same test & attack portion of data
    x_attack_train, x_attack_test= attack_split(x_attack, split_value)
    print("{} shape in the test set is {} after split of {}".format(attack, x_attack_test.shape,split))

    #these are OK like this for all layers
    y_attack_train=np.ones((x_attack_train.shape[0],1))
    y_attack_test=np.ones((x_attack_test.shape[0],1))

    
    #load test normal 156, 105, 20, 88
    x_test= load_test(layer)
    x_test, useless1, useless2, useless3=split_test_normal(x_test,x_attack_test.shape[0])#take the last x_attack_test.shape[0] element from test set

    #y_test is built the same for all three layers
    y_test = np.zeros((x_test.shape[0],1))
   
    
    #initialize classifier
    xgbC=xgb.XGBClassifier(nthread=CPU_THREADS,
                           use_label_encoder=False,
                           objective= 'binary:logistic',
                           eval_metric='logloss')
    #fit    
    print("now training")
    #xgbC.fit(linearize(np.vstack((x_train,x_attack_train))), np.vstack((y_train, y_attack_train)))
    #xgbC.save_model(MODEL_SAVE+str(attack)+'_'+str(layer)+'.model')
    xgbC.load_model(MODEL_SAVE+str(attack)+'_'+str(layer)+'.model')

    #predict on the test set and analysis of results
    print("now running predictions")
    final_preds=xgbC.predict(linearize(np.vstack((x_test, x_attack_test))))
    final_preds_proba=xgbC.predict_proba(linearize(np.vstack((x_test, x_attack_test))))

    #compute metrics
    
    compute_metrics_and_log(y_test, y_attack_test, 
                            final_preds, final_preds_proba, 
                            approach= "just one layer", METHOD="XGBoost only", split=split, layer=layer)
   
    #now repeat prediction on each attack
    #we need to recover the test split applied for the other attack
    for attack1 in ATTACK:
        #load attacks 
        x_attack =load_attack(attack1, layer)

        x_attack, useless1, useless2, useless3=define_attack_set_1(attack1, x_attack_test.shape[0], x_attack)        
        
        y_attack_test=np.ones((x_attack.shape[0],1))
        print("{} shape after split {}".format(attack1, x_attack.shape))
        final_preds=xgbC.predict(linearize(x_attack))
        final_preds_proba=xgbC.predict_proba(linearize(np.vstack((x_test,x_attack))))
        compute_metrics_and_log_attacks(y_test, y_attack_test, final_preds,
                                                             final_preds_proba,
                                                             approach= "just one layer",
                                                             METHOD="XGBoost only",
                                                             split=split, layer=layer)


We now use average pooling to reduce the size of layer 105. This can slightly improve performance if we reduce its size.

We try the following reduction parameters:

In [ ]:
parameters=[ 
    #C, W, H, stride1, stride2, stride3
#    (2, 3, 3, 2, 2, 2),
#    (2, 3, 3, 2, 2, 1),
#    (1, 6, 6, 2, 2, 1),
#    (2, 6, 6, 3, 3, 1),
#    (3, 6, 6, 2, 2, 2),
    (2, 4, 4, 2, 2, 1)
    ]

class NeuralNetwork(nn.Module):
    def __init__(self, C, H, W, stride1, stride2, stride3):
        super(NeuralNetwork, self).__init__()
        self.premodel_layer_1 = nn.Sequential(
            nn.AvgPool2d((H, W), stride=stride1)
        )
        self.premodel_layer_2 = nn.Sequential(
            nn.AvgPool2d((1, C), stride=(1,stride3))
        )
        
    def forward(self, out):
        out = self.premodel_layer_1(torch.tensor(out))
        out=torch.transpose(out, 1, 3)
        out = self.premodel_layer_2(out)
        out=out.detach().numpy()
        out=np.squeeze(out)
        return out


In [ ]:
def linearize_fgsm_deep(attack):
    if(attack=="fgsm_0156"):
        C, H, W, stride1, stride2, stride3= 2, 4, 4, 2, 2, 1
        pre_model=NeuralNetwork(C, W, H, stride1, stride2, stride3)
    #linearize deep_10
    elif(attack=="deep_10"):
        C, H, W, stride1, stride2, stride3= 3, 6, 6, 2, 2, 2
        pre_model=NeuralNetwork(C, W, H, stride1, stride2, stride3)
    return C, H, W, stride1, stride2, stride3, pre_model

def reduce_pre_model(x_train_105, x_test_105, x_attack_105, pre_model):
    x_train_105=pre_model(x_train_105)
    x_test_105=pre_model(x_test_105)
    x_attack_105=pre_model(x_attack_105)
    return x_train_105, x_test_105, x_attack_105

The following just prints the parameters that will be tested in the next block

From running the models, it will result that the following configuration should be selected for deep_10 and fgsm_0156:

C, W, H, stride1, stride2, stride3:  (2, 4, 4, 2, 2, 1)

This reduction brings layer 105 from (8, 8, 304) to (151, 3, 7) i.e., from 19456 features values to 3171. This reduction can improve detection on layer 105. So it will be used from now on.

The corresponding models will be created with the names

*deep_10_105_244221_reduced.model*

*fgsm_0156_105_244221_reduced.model*


In [ ]:
for layer, test_split, attack in list_combo:
    for param in parameters:
        if(layer != 105):
            continue
        print(attack, analyze_split(test_split, attack)[1], param)

In [ ]:
for layer, test_split, attack in list_combo:
    for param in parameters:
        if(layer != 105):
            continue
        split, split_value, split_index=analyze_split(test_split, attack)

        #load train 105
        x_train= load_train(layer)
        y_train = np.zeros((x_train.shape[0],1))

        #load attacks of the three layers
        x_attack=load_attack(attack, layer)
        print("{} shape of attacks in train + test  {}".format(attack, x_attack.shape))
    
        #split also attacks, to maintain the same test & attack portion of data
        x_attack_train, x_attack_test= attack_split(x_attack, split_value)
        print("{} shape in the test set is {} after split of {}".format(attack, x_attack_test.shape,split))

        #these are OK like this for all layers
        y_attack_train=np.ones((x_attack_train.shape[0],1))
        y_attack_test=np.ones((x_attack_test.shape[0],1))
        
        #load test normal 105
        x_test= load_test(layer)
        x_test, useless1, useless2, useless3=split_test_normal(x_test,x_attack_test.shape[0])

        #y_test is built the same for all three layers
        y_test = np.zeros((x_test.shape[0],1))



        #initialize classifier
        xgbC=xgb.XGBClassifier(nthread=CPU_THREADS,
                           use_label_encoder=False,
                           objective= 'binary:logistic',
                           eval_metric='logloss')

        C, W, H, stride1, stride2, stride3=param
        pre_model=NeuralNetwork(C, W, H, stride1, stride2, stride3)
        x_train=pre_model(x_train)
        x_attack_train=pre_model(x_attack_train)
        x_test=pre_model(x_test)
        x_attack_test=pre_model(x_attack_test)
        print("shape of hidden features after avgpool is {}".format(x_train.shape[1:]))
    
        #fit    
        print("now training")
        xgbC.fit(linearize(np.vstack((x_train,x_attack_train))), np.vstack((y_train, y_attack_train)))
        xgbC.save_model(MODEL_SAVE+str(attack)+"_"+str(layer)+"_"+
                        str(C)+str(W)+str(H)+str(stride1)+str(stride2)+str(stride3)+"_reduced.model")

        #predict on the test set and analysis of results
        print("now running predictions")
        final_preds=xgbC.predict(linearize(np.vstack((x_test, x_attack_test))))
        final_preds_proba=xgbC.predict_proba(linearize(np.vstack((x_test, x_attack_test))))

        #compute metrics
        compute_metrics_and_log(y_test, y_attack_test, 
                            final_preds, final_preds_proba, 
                            approach= "layer 105 with avg pool", METHOD="XGBoost only", split=split, layer=layer)
   
        #now repeat prediction on each attack
        #we need to recover the test split applied for the other attack
        for attack1 in ATTACK:
            #load attacks 
            
            x_attack =load_attack(attack1, layer)
            x_attack, useless1, useless2, useless3=define_attack_set_1(attack1, x_attack_test.shape[0],x_attack)        
        
            y_attack_test=np.ones((x_attack.shape[0],1))
            print("{} shape after split {}".format(attack1, x_attack.shape))
            x_attack=pre_model(x_attack)
            final_preds=xgbC.predict(linearize(x_attack))
            final_preds_proba=xgbC.predict_proba(linearize(np.vstack((x_test,x_attack))))
            compute_metrics_and_log_attacks(y_test, y_attack_test, final_preds, final_preds_proba,
                                        approach= "layer 105 with avg pool", METHOD="XGBoost only", 
                                        split=split, layer=layer)


We now apply the same reasoning and use average pooling to reduce the size of layer 20. This can slightly improve performance if we reduce its size.


Layer 20 has shape: (32, 32, 12) i.e., 12288 parameters

In [ ]:
for layer, test_split, attack in list_combo:
    for param in parameters:
        if(layer != 20):
            continue

        split, split_value, split_index=analyze_split(test_split, attack)

        #load train 20
        x_train= load_train(layer)
        y_train = np.zeros((x_train.shape[0],1))

        #load attacks of the three layers
        x_attack=load_attack(attack, layer)
        print("{} shape of attacks in train + test  {}".format(attack, x_attack.shape))
    
        #split also attacks, to maintain the same test & attack portion of data
        x_attack_train, x_attack_test= attack_split(x_attack, split_value)
        print("{} shape in the test set is {} after split of {}".format(attack, x_attack_test.shape,split))

        #these are OK like this for all layers
        y_attack_train=np.ones((x_attack_train.shape[0],1))
        y_attack_test=np.ones((x_attack_test.shape[0],1))
        
        #load test normal 20
        x_test= load_test(layer)
        x_test, useless1, useless2, useless3=split_test_normal(x_test,x_attack_test.shape[0])

        #y_test is built the same for all three layers
        y_test = np.zeros((x_test.shape[0],1))


        #initialize classifier
        xgbC=xgb.XGBClassifier(nthread=CPU_THREADS,
                           use_label_encoder=False,
                           objective= 'binary:logistic',
                           eval_metric='logloss')

        C, W, H, stride1, stride2, stride3=param
        pre_model=NeuralNetwork(C, W, H, stride1, stride2, stride3)
        x_train=pre_model(x_train)
        x_attack_train=pre_model(x_attack_train)
        x_test=pre_model(x_test)
        x_attack_test=pre_model(x_attack_test)
        print("shape of hidden features after avgpool is {}".format(x_train.shape[1:]))
    
        #fit    
        print("now training")
        xgbC.fit(linearize(np.vstack((x_train,x_attack_train))), np.vstack((y_train, y_attack_train)))
        xgbC.save_model(MODEL_SAVE+str(attack)+"_"+str(layer)+"_"+
                        str(C)+str(W)+str(H)+str(stride1)+str(stride2)+str(stride3)+"_reduced.model")

        #predict on the test set and analysis of results
        print("now running predictions")
        final_preds=xgbC.predict(linearize(np.vstack((x_test, x_attack_test))))
        final_preds_proba=xgbC.predict_proba(linearize(np.vstack((x_test, x_attack_test))))

        #compute metrics
        compute_metrics_and_log(y_test, y_attack_test, 
                            final_preds, final_preds_proba, 
                            approach= "layer 20 with avg pool", METHOD="XGBoost only", split=split, layer=layer)
   
        #now repeat prediction on each attack
        #we need to recover the test split applied for the other attack
        for attack1 in ATTACK:
            #load attacks 
            x_attack =load_attack(attack1, layer)
            x_attack, useless1, useless2, useless3=define_attack_set_1(attack1, x_attack_test.shape[0],x_attack)        
        
            y_attack_test=np.ones((x_attack.shape[0],1))
            print("{} shape after split {}".format(attack1, x_attack.shape))
            x_attack=pre_model(x_attack)
            final_preds=xgbC.predict(linearize(x_attack))
            final_preds_proba=xgbC.predict_proba(linearize(np.vstack((x_test,x_attack))))
            compute_metrics_and_log_attacks(y_test, y_attack_test, final_preds, final_preds_proba,
                                        approach= "layer 20 with avg pool", METHOD="XGBoost only", 
                                        split=split, layer=layer)


We now use average pooling to reduce the size of layer 88. This can slightly improve performance if we reduce its size.

Layer 88 has shape: (16, 16, 12), that is 3072 parameters.

In [ ]:
for layer, test_split, attack in list_combo:
    for param in parameters:
        if(layer != 88):
            continue

        split, split_value, split_index=analyze_split(test_split, attack)

        #load train 88
        x_train= load_train(layer)
        y_train = np.zeros((x_train.shape[0],1))

        #load attacks 
        x_attack=load_attack(attack, layer)
        print("{} shape of attacks in train + test  {}".format(attack, x_attack.shape))
    
        #split also attacks, to maintain the same test & attack portion of data
        x_attack_train, x_attack_test= attack_split(x_attack, split_value)
        print("{} shape in the test set is {} after split of {}".format(attack, x_attack_test.shape,split))

        #these are OK like this for all layers
        y_attack_train=np.ones((x_attack_train.shape[0],1))
        y_attack_test=np.ones((x_attack_test.shape[0],1))
        
        #load test normal 88
        x_test= load_test(layer)
        x_test, useless1, useless2, useless3=split_test_normal(x_test,x_attack_test.shape[0])

        #y_test is built the same for all three layers
        y_test = np.zeros((x_test.shape[0],1))

        #initialize classifier
        xgbC=xgb.XGBClassifier(nthread=CPU_THREADS,
                           use_label_encoder=False,
                           objective= 'binary:logistic',
                           eval_metric='logloss')

        C, W, H, stride1, stride2, stride3=param
        pre_model=NeuralNetwork(C, W, H, stride1, stride2, stride3)
        x_train=pre_model(x_train)
        x_attack_train=pre_model(x_attack_train)
        x_test=pre_model(x_test)
        x_attack_test=pre_model(x_attack_test)
        print("shape of hidden features after avgpool is {}".format(x_train.shape[1:]))
    
        #fit    
        print("now training")
        xgbC.fit(linearize(np.vstack((x_train,x_attack_train))), np.vstack((y_train, y_attack_train)))
        xgbC.save_model(MODEL_SAVE+str(attack)+"_"+str(layer)+"_"+
                        str(C)+str(W)+str(H)+str(stride1)+str(stride2)+str(stride3)+"_reduced.model")

        #predict on the test set and analysis of results
        print("now running predictions")
        final_preds=xgbC.predict(linearize(np.vstack((x_test, x_attack_test))))
        final_preds_proba=xgbC.predict_proba(linearize(np.vstack((x_test, x_attack_test))))

        #compute metrics
        compute_metrics_and_log(y_test, y_attack_test, 
                            final_preds, final_preds_proba, 
                            approach= "layer 88 with avg pool", METHOD="XGBoost only", split=split, layer=layer)
   
        #now repeat prediction on each attack
        #we need to recover the test split applied for the other attack
        for attack1 in ATTACK:
            #load attacks 
            x_attack =load_attack(attack1, layer)
            x_attack, useless1, useless2, useless3=define_attack_set_1(attack1, x_attack_test.shape[0],x_attack)        
        
            y_attack_test=np.ones((x_attack.shape[0],1))
            print("{} shape after split {}".format(attack1, x_attack.shape))
            x_attack=pre_model(x_attack)
            final_preds=xgbC.predict(linearize(x_attack))
            final_preds_proba=xgbC.predict_proba(linearize(np.vstack((x_test,x_attack))))
            compute_metrics_and_log_attacks(y_test, y_attack_test, final_preds, final_preds_proba,
                                        approach= "layer 88 with avg pool", METHOD="XGBoost only", 
                                        split=split, layer=layer)


Analysing the log, we can  observe that the preferred avgpool for 105 is the configuration:

- *deep_10_105_244221_reduced.model*
- *fgsm_0156_105_244221_reduced.model*

For layer 20:

- there is no actual improvement, so no reduced model is used

For layer 88, it is: 

- there is no actual improvement, so no reduced model is use


**We will use the reduced models we identfied, from now on, instead than the original model.**

Now we try 2 layers in stacking: 156 and 105, the best 2 layers we have.

In [ ]:
def initialize_classifiers():
    a=xgb.XGBClassifier(nthread=CPU_THREADS,use_label_encoder=False,objective='binary:logistic',eval_metric='logloss')
    b=xgb.XGBClassifier(nthread=CPU_THREADS,use_label_encoder=False,objective='binary:logistic',eval_metric='logloss')
    c=xgb.XGBClassifier(nthread=CPU_THREADS,use_label_encoder=False,objective='binary:logistic',eval_metric='logloss')
    d=xgb.XGBClassifier(nthread=CPU_THREADS,use_label_encoder=False,objective='binary:logistic',eval_metric='logloss')    
    e=xgb.XGBClassifier(nthread=CPU_THREADS,use_label_encoder=False,objective='binary:logistic',eval_metric='logloss')    
    return a, b, c, d, e

list_combo= [(y,d)
             for y in TEST_SPLIT 
             for d in MAIN_ATTACK_LIST
            ]

In [ ]:
def predict_proba_xgb(layer156,xgb_156,layer105=None, xgb_105=None,layer20=None,xgb_20=None, 
                      layer88=None, xgb_88=None):
    meta_156=xgb_156.predict_proba(linearize(layer156))
    meta_105=None
    meta_20=None
    meta_88=None #just to initialize vars
    if(layer105 is not None):
        meta_105=xgb_105.predict_proba(linearize(layer105))
    if(layer20 is not None):
        meta_20=xgb_20.predict_proba(linearize(layer20))
    if(layer88 is not None):
        meta_88=xgb_88.predict_proba(linearize(layer88))
    return meta_156, meta_105, meta_20, meta_88

#reduces layer 105 of C 2 W 4 H 4 stride1 2 stride2 2 stride3 1  
def avgpool_reduction(a,b=None,c=None, d=None):
    C, W, H, stride1, stride2, stride3=2, 4, 4, 2, 2, 1
    pre_model=NeuralNetwork(C, W, H, stride1, stride2, stride3)
    a=pre_model(a)
    if(b is not None):
        b=pre_model(b)
    if(c is not None):
        c=pre_model(c)
    if(d is not None):
        d=pre_model(d)
    return a,b,c,d

In [ ]:
for test_split, attack in list_combo:
    split, split_value, split_index=analyze_split(test_split, attack)
    #and for all attacks
    x_attack_156, x_attack_105, useless1, useless2=load_4_layers_attack(attack, 156, 105)
    #split attacks as usual, to maintain the same test & attack portion of data
    x_attack_train_156, x_attack_test_156= attack_split(x_attack_156, split_value)
    x_attack_train_105, x_attack_test_105= attack_split(x_attack_105, split_value)
    #these are OK like this for all layers
    y_attack_train=np.ones((x_attack_train_156.shape[0],1))
    y_attack_test=np.ones((x_attack_test_156.shape[0],1))
    
    #load train, create y_train
    x_train_156,x_train_105,useless1, useless2= load_4_layers_train(156, 105)
    y_train = np.zeros((x_train_156.shape[0],1))
    #same for test
    x_test_156,x_test_105, useless1, useless2= load_4_layers_test(156, 105)
    x_test_156,x_test_105, useless1, useless2=split_test_normal(x_test_156, x_attack_test_156.shape[0], x_test_105)
    y_test = np.zeros((x_test_156.shape[0],1))

    #initialize all classifiers (base and stacker)
    xgb_TOT, xgb_156, xgb_105, useless1, useless2=initialize_classifiers()
    
    xgb_156.load_model(MODEL_SAVE+str(attack)+"_"+str(156)+".model")
    xgb_105.load_model(MODEL_SAVE+str(attack)+"_105_244221_reduced.model")
    
    x_train_105,x_attack_train_105,x_attack_test_105, x_test_105=avgpool_reduction(x_train_105,x_attack_train_105,
                                                                                   x_attack_test_105, x_test_105)
    #prediction made by base classifiers (create meta-data for stacking)
    meta_156, meta_105, useless1, useless2 =predict_proba_xgb(np.vstack((x_train_156,x_attack_train_156)),xgb_156,
                                                              np.vstack((x_train_105,x_attack_train_105)),xgb_105)

    #fit the meta-learner    
    print("training meta classifier")
    xgb_TOT.fit(np.column_stack((meta_156, meta_105)), np.vstack((y_train, y_attack_train)))
    xgb_TOT.save_model(MODEL_SAVE+"xgboost_stacker_156_105.model")

    #predict on the test set and analysis of results
    print("now running predictions")
    meta_156_test_normal, meta_105_test_normal, useless1, useless2=predict_proba_xgb(x_test_156,xgb_156,
                                                                                      x_test_105,xgb_105)
    
    meta_156_test_attack,meta_105_test_attack,useless1, useless2=predict_proba_xgb(x_attack_test_156,xgb_156,
                                                                                x_attack_test_105,xgb_105)
    
    meta_test_normal=np.column_stack((meta_156_test_normal, meta_105_test_normal))
    meta_test_attack=np.column_stack((meta_156_test_attack,meta_105_test_attack))
    
    final_preds=xgb_TOT.predict(np.vstack((meta_test_normal, meta_test_attack)))

    final_preds_proba=xgb_TOT.predict_proba(np.vstack((meta_test_normal, meta_test_attack)))

    #compute metrics
    compute_metrics_and_log(y_test, y_attack_test, 
                final_preds, final_preds_proba, 
                approach= "stacking of 156 and 105", METHOD="XGBoost only", split=split, layer=None)


    #now repeat prediction on each attack
    #we would need to recover the test split from the other one
    for attack1 in ATTACK:
        #load attacks 
        x_attack_156, x_attack_105, useless1, useless2 =load_4_layers_attack(attack1, 156, 105)
        x_attack_156, x_attack_105, useless1, useless2=define_attack_set_1(attack1,
                                                                         x_attack_test_156.shape[0],
                                                                         x_attack_156,
                                                                         x_attack_105)

        x_attack_105,useless1,useless2, useless3=avgpool_reduction(x_attack_105)
        
        y_attack_test=np.ones((x_attack_156.shape[0],1))
        meta_156,meta_105,useless1, useless2=predict_proba_xgb(x_attack_156,xgb_156,x_attack_105, xgb_105)
    
        meta_test_attack=np.column_stack((meta_156, meta_105))
        final_preds=xgb_TOT.predict(meta_test_attack)
        final_preds_proba=xgb_TOT.predict_proba(np.vstack((meta_test_normal,meta_test_attack)))

        compute_metrics_and_log_attacks(y_test, y_attack_test, 
                    final_preds, final_preds_proba, 
                    approach= "stacking of 156 and 105", METHOD="XGBoost only", split=split, layer=None)

Now we try 3 layers in stacking: 156, 105, 20, the best 3 layers we have.

In [ ]:
for test_split, attack in list_combo:

    split, split_value, split_index=analyze_split(test_split, attack)
    #load train, create y_train
    x_train_156,x_train_105,x_train_20, useless2= load_4_layers_train(156, 105, 20)
    y_train = np.zeros((x_train_156.shape[0],1))
    x_attack_156, x_attack_105, x_attack_20, useless2=load_4_layers_attack(attack, 156, 105, 20)
    #split attacks as usual, to maintain the same test & attack portion of data
    x_attack_train_156, x_attack_test_156= attack_split(x_attack_156, split_value)
    x_attack_train_105, x_attack_test_105= attack_split(x_attack_105, split_value)
    x_attack_train_20, x_attack_test_20= attack_split(x_attack_20, split_value)
    #these are OK like this for all layers
    y_attack_train=np.ones((x_attack_train_156.shape[0],1))
    y_attack_test=np.ones((x_attack_test_156.shape[0],1))
    
    
    #same for test
    x_test_156,x_test_105, x_test_20, useless2= load_4_layers_test(156, 105, 20)
    x_test_156,x_test_105, x_test_20, useless2=split_test_normal(x_test_156, x_attack_test_156.shape[0], x_test_105, x_test_20)
    y_test = np.zeros((x_test_156.shape[0],1))
    #and for all attacks

    #initialize all classifiers (base and stacker)
    xgb_TOT, xgb_156, xgb_105, xgb_20, useless2=initialize_classifiers()
    
    xgb_156.load_model(MODEL_SAVE+str(attack)+"_"+str(156)+".model")
    xgb_105.load_model(MODEL_SAVE+str(attack)+"_105_244221_reduced.model")
    xgb_20.load_model(MODEL_SAVE+str(attack)+"_"+str(20)+".model")
    
    x_train_105,x_attack_train_105,x_attack_test_105, x_test_105=avgpool_reduction(x_train_105,x_attack_train_105,x_attack_test_105, x_test_105)
    
    #prediction made by base classifiers (create meta-data for stacking)
    meta_156, meta_105, meta_20, useless2 =predict_proba_xgb(np.vstack((x_train_156,x_attack_train_156)),xgb_156,
                                                              np.vstack((x_train_105,x_attack_train_105)),xgb_105,
                                                              np.vstack((x_train_20,x_attack_train_20)),xgb_20,
                                                            )

    #fit the meta-learner    
    print("training meta classifier")
    xgb_TOT.fit(np.column_stack((meta_156, meta_105, meta_20)), np.vstack((y_train, y_attack_train)))
    xgb_TOT.save_model(MODEL_SAVE+"xgboost_stacker_156_105_20.model")

    #predict on the test set and analysis of results
    print("now running predictions")
    meta_156_test_normal, meta_105_test_normal, meta_20_test_normal, useless2=predict_proba_xgb(x_test_156,xgb_156,
                                                                                      x_test_105,xgb_105,
                                                                                      x_test_20,xgb_20)
    
    meta_156_test_attack,meta_105_test_attack,meta_20_test_attack, useless2=predict_proba_xgb(x_attack_test_156,xgb_156,
                                                                                x_attack_test_105,xgb_105,
                                                                                x_attack_test_20,xgb_20)
    
    meta_test_normal=np.column_stack((meta_156_test_normal, meta_105_test_normal,meta_20_test_normal))
    meta_test_attack=np.column_stack((meta_156_test_attack,meta_105_test_attack,meta_20_test_attack))
    
    final_preds=xgb_TOT.predict(np.vstack((meta_test_normal, meta_test_attack)))

    final_preds_proba=xgb_TOT.predict_proba(np.vstack((meta_test_normal, meta_test_attack)))

    #compute metrics
    compute_metrics_and_log(y_test, y_attack_test, 
                final_preds, final_preds_proba, 
                approach= "stacking of 156-105-20", METHOD="XGBoost only", split=split, layer=None)


    #now repeat prediction on each attack
    #we would need to recover the test split from the other one
    for attack1 in ATTACK:
        #load attacks of the three layers
        x_attack_156, x_attack_105, x_attack_20, useless2 =load_4_layers_attack(attack1, 156, 105, 20)

        x_attack_156, x_attack_105, x_attack_20, useless2=define_attack_set_1(attack1,
                                                                            x_attack_test_156.shape[0],
                                                                            x_attack_156,
                                                                            x_attack_105,
                                                                            x_attack_20)
        x_attack_105, useless1,useless2, useless3=avgpool_reduction(x_attack_105)
        
        y_attack_test=np.ones((x_attack_156.shape[0],1))

        meta_156,meta_105,meta_20, useless2=predict_proba_xgb(x_attack_156,xgb_156,x_attack_105, xgb_105,x_attack_20, xgb_20)
    
        meta_test_attack=np.column_stack((meta_156, meta_105, meta_20))
        final_preds=xgb_TOT.predict(meta_test_attack)
        final_preds_proba=xgb_TOT.predict_proba(np.vstack((meta_test_normal,meta_test_attack)))

        compute_metrics_and_log_attacks(y_test, y_attack_test, 
                    final_preds, final_preds_proba, 
                    approach= "stacking of 156-105-20", METHOD="XGBoost only", split=split, layer=None)

Now we try 4 layers in stacking: 156, 105, 20, 88

We should notice a very slight improvement.

In [ ]:
for test_split, attack in list_combo:
    split, split_value, split_index=analyze_split(test_split, attack)
    #load train, create y_train
    x_train_156,x_train_105,x_train_20, x_train_88= load_4_layers_train(156, 105, 20, 88)
    y_train = np.zeros((x_train_156.shape[0],1))
#and for all attacks
    x_attack_156, x_attack_105, x_attack_20,x_attack_88=load_4_layers_attack(attack, 156, 105, 20, 88)
    #split attacks as usual, to maintain the same test & attack portion of data
    x_attack_train_156, x_attack_test_156= attack_split(x_attack_156, split_value)
    x_attack_train_105, x_attack_test_105= attack_split(x_attack_105, split_value)
    x_attack_train_20, x_attack_test_20= attack_split(x_attack_20, split_value)
    x_attack_train_88, x_attack_test_88= attack_split(x_attack_88, split_value)
    #these are OK like this for all layers
    y_attack_train=np.ones((x_attack_train_156.shape[0],1))
    y_attack_test=np.ones((x_attack_test_156.shape[0],1))

    
    #same for test
    x_test_156,x_test_105, x_test_20, x_test_88= load_4_layers_test(156, 105, 20,88)
    x_test_156,x_test_105, x_test_20, x_test_88=split_test_normal(x_test_156, 
                                                                  x_attack_test_156.shape[0],
                                                                  x_test_105, x_test_20, 
                                                                  x_test_88)
    y_test = np.zeros((x_test_156.shape[0],1))
    
    #initialize all classifiers (base and stacker)
    xgb_TOT, xgb_156, xgb_105, xgb_20,xgb_88=initialize_classifiers()
    
    xgb_156.load_model(MODEL_SAVE+str(attack)+"_"+str(156)+".model")
    xgb_105.load_model(MODEL_SAVE+str(attack)+"_105_244221_reduced.model")
    xgb_20.load_model(MODEL_SAVE+str(attack)+"_"+str(20)+".model")
    xgb_88.load_model(MODEL_SAVE+str(attack)+"_"+str(88)+".model")

    #avgpool on 105 as usual
    x_train_105,x_attack_train_105,x_attack_test_105, x_test_105=avgpool_reduction(x_train_105,x_attack_train_105,x_attack_test_105, x_test_105)
    #prediction made by base classifiers (create meta-data for stacking)
    meta_156, meta_105, meta_20, meta_88 =predict_proba_xgb(np.vstack((x_train_156,x_attack_train_156)),xgb_156,
                                                              np.vstack((x_train_105,x_attack_train_105)),xgb_105,
                                                              np.vstack((x_train_20,x_attack_train_20)),xgb_20,
                                                              np.vstack((x_train_88,x_attack_train_88)),xgb_88,
                                                           )

    #fit the meta-learner    
    print("training meta classifier")
    xgb_TOT.fit(np.column_stack((meta_156, meta_105, meta_20,meta_88)), np.vstack((y_train, y_attack_train)))
    xgb_TOT.save_model(MODEL_SAVE+"xgboost_stacker_156_105_20_88.model")

    #predict on the test set and analysis of results
    print("now running predictions")
    meta_156_test_normal, meta_105_test_normal, meta_20_test_normal, meta_88_test_normal=predict_proba_xgb(x_test_156,xgb_156,
                                                                                      x_test_105,xgb_105,
                                                                                      x_test_20,xgb_20,
                                                                                      x_test_88,xgb_88)
    
    meta_156_test_attack,meta_105_test_attack,meta_20_test_attack, meta_88_test_attack=predict_proba_xgb(x_attack_test_156,xgb_156,
                                                                                x_attack_test_105,xgb_105,
                                                                                x_attack_test_20,xgb_20,
                                                                                x_attack_test_88,xgb_88)
    
    meta_test_normal=np.column_stack((meta_156_test_normal, meta_105_test_normal,meta_20_test_normal,meta_88_test_normal ))
    meta_test_attack=np.column_stack((meta_156_test_attack,meta_105_test_attack,meta_20_test_attack,meta_88_test_attack))
    
    final_preds=xgb_TOT.predict(np.vstack((meta_test_normal, meta_test_attack)))

    final_preds_proba=xgb_TOT.predict_proba(np.vstack((meta_test_normal, meta_test_attack)))

    #compute metrics
    compute_metrics_and_log(y_test, y_attack_test, 
                final_preds, final_preds_proba, 
                approach= "stacking of 156-105-20-88", METHOD="XGBoost only", split=split, layer=None)

    #now repeat prediction on each attack
    #we would need to recover the test split from the other one
    for attack1 in ATTACK:
        #load attacks of the three layers
        x_attack_156, x_attack_105, x_attack_20, x_attack_88 =load_4_layers_attack(attack1, 156, 105, 20,88)

        x_attack_156, x_attack_105, x_attack_20, x_attack_88=define_attack_set_1(attack1,
                                                                               x_attack_test_156.shape[0],
                                                                               x_attack_156, x_attack_105, x_attack_20,x_attack_88)
        
        x_attack_105, useless1,useless2, useless3=avgpool_reduction(x_attack_105)

        y_attack_test=np.ones((x_attack_156.shape[0],1))
        meta_156,meta_105,meta_20,meta_88=predict_proba_xgb(x_attack_156,xgb_156,x_attack_105, xgb_105,x_attack_20, xgb_20,x_attack_88, xgb_88)
    
        meta_test_attack=np.column_stack((meta_156, meta_105, meta_20, meta_88))
        final_preds=xgb_TOT.predict(meta_test_attack)
        final_preds_proba=xgb_TOT.predict_proba(np.vstack((meta_test_normal,meta_test_attack)))

        compute_metrics_and_log_attacks(y_test, y_attack_test, 
                    final_preds, final_preds_proba, 
                    approach= "stacking of 156-105-20-88", METHOD="XGBoost only", split=split, layer=None)

We compare results with XGBoost using another method for tabular data. We select fastai, because it has some nice preprocessing activities. 

First, we compute fastai on each layer.


In [ ]:
def trainFastai_one_layer(x_train_156,x_attack_train_156,y_train, y_attack_train):

    df,cont_names, cat_names, label =create_dataframe(linearize(np.vstack((x_train_156,x_attack_train_156))),
                                                      np.vstack((y_train, y_attack_train)))
    split = RandomSplitter(valid_pct=VAL_SPLIT, seed=42)(range_of(df))    
    to = TabularPandas(df,procs=[Categorify, FillMissing, Normalize],
                       cat_names = cat_names,
                       cont_names = cont_names,
                       y_names=label,
                       splits=split,
                       reduce_memory=True)

    dls_156 = to.dataloaders(bs=BATCH_SIZE)#, device=torch.device('cuda'))
    fastai_156=tabular_learner(dls_156, metrics=accuracy)
    fastai_156.fit_one_cycle(EPOCH, cbs=[ShowGraphCallback(),
                                         EarlyStoppingCallback(monitor='valid_loss', min_delta=0.001, patience=3)])
    
    return fastai_156, dls_156


def predict_proba_fastai(x_train, fastai_156, dls_156):
    df1=pd.DataFrame(linearize(x_train))
    dl1 = dls_156.test_dl(df1, bs=BATCH_SIZE)#, device=torch.device('cuda')) # apply transforms
    meta_156_fastai,  _ = fastai_156.get_preds(dl=dl1) # get prediction
    meta_156_fastai=meta_156_fastai.numpy()
    return meta_156_fastai


def predict_fastai(x_train, fastai_156, dls_156):
    df1=pd.DataFrame(linearize(x_train))
    dl1 = dls_156.test_dl(df1, bs=BATCH_SIZE)#, device=torch.device('cuda')) # apply transforms
    meta_156_fastai,  _ = fastai_156.get_preds(dl=dl1) # get prediction
    meta_156_fastai=meta_156_fastai.numpy()
    return meta_156_fastai


In [ ]:
list_combo= [(L, y,d)
             for L in TOP_LAYERS 
             for y in TEST_SPLIT 
             for d in MAIN_ATTACK_LIST
            ]

VAL_SPLIT=0.2
import pickle

In [ ]:
for layer, test_split, attack in list_combo:
    %reset_selective -f to, dls, learn, split
    import fastai
    from fastai.tabular.all import *
    split, split_value, split_index=analyze_split(test_split, attack)

    #load train 156, 105, 20, 88
    x_train= load_train(layer)
    y_train = np.zeros((x_train.shape[0],1))

    #load test normal 156, 105, 20, 88
    x_test= load_test(layer)
    x_test, useless1, useless2, useless3=split_test_normal(x_test,split_index)

    #y_test is built the same for all three layers
    y_test = np.zeros((x_test.shape[0],1))

    #load attacks of the layers
    x_attack=load_attack(attack, layer)
    print("{} shape of attacks in train + test  {}".format(attack, x_attack.shape))
    
    #split also attacks, to maintain the same test & attack portion of data
    x_attack_train, x_attack_test= attack_split(x_attack, split_value)
    print("{} shape in the test set is {} after split of {}".format(attack, x_attack_test.shape,split))

    #these are OK like this for all layers
    y_attack_train=np.ones((x_attack_train.shape[0],1))
    y_attack_test=np.ones((x_attack_test.shape[0],1))

    #avgpool on 105
    if(layer==105):
        x_train,x_attack_train,x_attack_test,x_test=avgpool_reduction(x_train,x_attack_train,x_attack_test,x_test)

    #train fastai and save model
    fastai_model, dls_model=trainFastai_one_layer(x_train,x_attack_train,y_train, y_attack_train)
    fastai_model.save(MODEL_SAVE+"fastai_"+str(attack)+"_"+str(layer))
    with open(MODEL_SAVE+"dls_fastai_"+str(attack)+"_"+str(layer), 'wb') as pickle_file:
        pickle.dump(dls_model, pickle_file, protocol=4.0)


    #predict on the test set and analysis of results
    print("now running predictions")
    final_preds_proba=predict_proba_fastai(np.vstack((x_test, x_attack_test)), fastai_model, dls_model)
    final_preds = np.argmax(final_preds_proba, axis=1)

    #compute metrics
    compute_metrics_and_log(y_test, y_attack_test, 
                            final_preds, final_preds_proba, 
                            approach= "just one layer", METHOD="FastAI only", split=split, layer=layer)
   
    #now repeat prediction on each attack
    #we need to recover the test split applied for the other attack
    for attack1 in ATTACK:
        #load attacks of the layer
        x_attack =load_attack(attack1, layer)
        if(layer==105):
            x_attack, useless1, useless2, useless3=avgpool_reduction(x_attack)

        x_attack, useless1, useless2, useless3=define_attack_set_1(test_split, attack1, split_value, split_index,x_attack)        
        
        y_attack_test=np.ones((x_attack.shape[0],1))
        print("{} shape after split {}".format(attack1, x_attack.shape))
        final_preds_proba=predict_proba_fastai(np.vstack((x_test,x_attack)),fastai_model, dls_model)
        final_preds=predict_proba_fastai(x_attack,fastai_model, dls_model)
        final_preds = np.argmax(final_preds, axis=1)
        
        compute_metrics_and_log_attacks(y_test, y_attack_test, final_preds, final_preds_proba,
                                        approach= "just one layer", METHOD="FastAI only", split=split, layer=layer)

In [ ]:
list_combo= [(y,d)
             for y in TEST_SPLIT 
             for d in MAIN_ATTACK_LIST
            ]

Compute the time it takes to XGBoost to perform the prediction with 4 stacking layers

In [ ]:
import time
for test_split, attack in list_combo:
    start_time = time.time()
    
    x_test_156,x_test_105, x_test_20, x_test_88= load_4_layers_test(156, 105, 20,88)
    y_test = np.zeros((x_test_156.shape[0],1))
    #and all attacks
    x_attack_156, x_attack_105, x_attack_20,x_attack_88=load_4_layers_attack(attack, 156, 105, 20, 88)
    y_attack_test=np.ones((x_attack_156.shape[0],1))

    #initialize all classifiers (base and stacker)
    xgb_TOT, xgb_156, xgb_105, xgb_20,xgb_88=initialize_classifiers()
    
    xgb_156.load_model(MODEL_SAVE+str(attack)+"_"+str(156)+".model")
    xgb_105.load_model(MODEL_SAVE+str(attack)+"_105_244221_reduced.model")
    xgb_20.load_model(MODEL_SAVE+str(attack)+"_"+str(20)+".model")
    xgb_88.load_model(MODEL_SAVE+str(attack)+"_"+str(88)+".model")
    print("loading meta classifier")
    xgb_TOT.load_model(MODEL_SAVE+"xgboost_stacker_156_105_20_88.model")

    #prepare data
    x_test_105=np.vstack((x_test_105, x_attack_105))
    x_test_88=np.vstack((x_test_88, x_attack_88))
    x_test_20=np.vstack((x_test_20, x_attack_20))
    x_test_156=np.vstack((x_test_156, x_attack_156))
    
    print("computing performance to predict on {} images".format(x_test_105.shape[0]))
    print("for attack: {} ".format(attack))
    #predict on the test set and analysis of results
    x_test_105,a,b,c=avgpool_reduction(x_test_105)
    meta_156=xgb_156.predict_proba(linearize(x_test_156))
    meta_105=xgb_105.predict_proba(linearize(x_test_105))
    meta_20=xgb_20.predict_proba(linearize(x_test_20))
    meta_88=xgb_88.predict_proba(linearize(x_test_88))

    xgb_TOT.predict(np.column_stack((meta_156, meta_105, meta_20, meta_88)))
    finish_time=time.time()
    
    print("--- %s seconds ---" % (finish_time - start_time))